## Dependency set up

In [1]:
# set the environment path to find Recommenders
%load_ext autoreload
%autoreload 2

import sys

import pyspark
from pyspark.ml.recommendation import ALS
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, FloatType, IntegerType, LongType, StructType, StructField
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover
from pyspark.ml.feature import HashingTF, CountVectorizer, VectorAssembler

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.utils.notebook_utils import is_jupyter
from recommenders.datasets.spark_splitters import spark_random_split
from recommenders.evaluation.spark_evaluation import SparkRatingEvaluation, SparkRankingEvaluation, SparkDiversityEvaluation
from recommenders.utils.spark_utils import start_or_get_spark

from pyspark.sql.window import Window
import pyspark.sql.functions as F

import numpy as np
import pandas as pd

print("System version: {}".format(sys.version))
print("Spark version: {}".format(pyspark.__version__))

System version: 3.7.0 (default, Oct  9 2018, 10:31:47) 
[GCC 7.3.0]
Spark version: 2.4.8


### Set the default parameters.

In [2]:
# top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

# user, item column names
COL_USER="UserId"
COL_ITEM="MovieId"
COL_RATING="Rating"

## 1. Set up Spark context
The following settings work well for debugging locally on VM - change when running on a cluster. We set up a giant single executor with many threads and specify memory cap.

In [3]:
spark = start_or_get_spark("ALS PySpark", memory="16g")

spark.conf.set("spark.sql.crossJoin.enabled", "true")

## 2. Download the MovieLens dataset

Note can set different size above

In [4]:
# Note: The DataFrame-based API for ALS currently only supports integers for user and item ids.
schema = StructType(
    (
        StructField(COL_USER, IntegerType()),
        StructField(COL_ITEM, IntegerType()),
        StructField(COL_RATING, FloatType()),
        StructField("Timestamp", LongType()),
    )
)

data = movielens.load_spark_df(spark, size=MOVIELENS_DATA_SIZE, schema=schema, title_col="title", genres_col="genres")
data.show()

100%|██████████| 4.81k/4.81k [00:00<00:00, 13.9kKB/s]


+-------+------+------+---------+--------------------+------+
|MovieId|UserId|Rating|Timestamp|               title|genres|
+-------+------+------+---------+--------------------+------+
|     26|   138|   5.0|879024232|Brothers McMullen...|Comedy|
|     26|   224|   3.0|888104153|Brothers McMullen...|Comedy|
|     26|    18|   4.0|880129731|Brothers McMullen...|Comedy|
|     26|   222|   3.0|878183043|Brothers McMullen...|Comedy|
|     26|    43|   5.0|883954901|Brothers McMullen...|Comedy|
|     26|   201|   4.0|884111927|Brothers McMullen...|Comedy|
|     26|   299|   4.0|878192601|Brothers McMullen...|Comedy|
|     26|    95|   3.0|880571951|Brothers McMullen...|Comedy|
|     26|    89|   3.0|879459909|Brothers McMullen...|Comedy|
|     26|   361|   3.0|879440941|Brothers McMullen...|Comedy|
|     26|   194|   3.0|879522240|Brothers McMullen...|Comedy|
|     26|   391|   5.0|877399745|Brothers McMullen...|Comedy|
|     26|   345|   3.0|884993555|Brothers McMullen...|Comedy|
|     26

### Split the data using the Spark random splitter provided in utilities

In [5]:
train_df, test_df = spark_random_split(data.select(COL_USER, COL_ITEM, COL_RATING), ratio=0.75, seed=123)
print ("N train_df", train_df.cache().count())
print ("N test_df", test_df.cache().count())

N train_df 75066
N test_df 24934


### Get all possible user-item pairs
Note: We assume that training data contains all users and all catalog items.

In [6]:
users = train_df.select(COL_USER).distinct()
items = train_df.select(COL_ITEM).distinct()
user_item = users.crossJoin(items)

In [10]:
print(users.count())
print(items.count())
print(user_item.count())

943
1633
1539919


## 3. Train the ALS model on the training data, and get the top-k recommendations for our testing data

To predict movie ratings, we use the rating data in the training set as users' explicit feedback. The hyperparameters used in building the model are referenced from here. We do not constrain the latent factors (nonnegative = False) in order to allow for both positive and negative preferences towards movies. Timing will vary depending on the machine being used to train.

<b>Note</b> on SCC with gpu it takes 2 sec.

In [7]:
header = {
    "userCol": COL_USER,
    "itemCol": COL_ITEM,
    "ratingCol": COL_RATING,
}


##could modify and have another algo here
als = ALS(
    rank=10,
    maxIter=15,
    implicitPrefs=False,
    regParam=0.05,
    coldStartStrategy='drop',
    nonnegative=False,
    seed=42,
    **header
)

Note here the model object is generated:

In [8]:
with Timer() as train_time:
    model = als.fit(train_df)

print("Took {} seconds for training.".format(train_time.interval))

Took 2.8200885746628046 seconds for training.


In the movie recommendation use case, recommending movies that have been rated by the users does not make sense. Therefore, the rated movies are removed from the recommended items.

In order to achieve this, we recommend all movies to all users, and then remove the user-movie pairs that exist in the training dataset.

In [9]:
# Score all user-item pairs
dfs_pred = model.transform(user_item)

# Remove seen items.
dfs_pred_exclude_train = dfs_pred.alias("pred").join(
    train_df.alias("train"),
    (dfs_pred[COL_USER] == train_df[COL_USER]) & (dfs_pred[COL_ITEM] == train_df[COL_ITEM]),
    how='outer'
)

top_all = dfs_pred_exclude_train.filter(dfs_pred_exclude_train["train.Rating"].isNull()) \
    .select('pred.' + COL_USER, 'pred.' + COL_ITEM, 'pred.' + "prediction")

print(top_all.count())
    
window = Window.partitionBy(COL_USER).orderBy(F.col("prediction").desc())    
top_k_reco = top_all.select("*", F.row_number().over(window).alias("rank")).filter(F.col("rank") <= TOP_K).drop("rank")
 
print(top_k_reco.count())

1464853
9430


## 4. Random Recommender - For comparison
We define a recommender which randomly recommends unseen items to each user.

In [11]:
# random recommender
window = Window.partitionBy(COL_USER).orderBy(F.rand()) # note understand better

# randomly generated recommendations for each user
pred_df = (
  train_df
  # join training data with all possible user-item pairs (seen in training)
  .join(user_item,
        on=[COL_USER, COL_ITEM],
        how="right"
  )
  # get user-item pairs that were not seen in the training data
  .filter(F.col(COL_RATING).isNull())
  # count items for each user (randomly sorting them)
  .withColumn("score", F.row_number().over(window))
  # get the top k items per user
  .filter(F.col("score") <= TOP_K)
  .drop(COL_RATING)
)

## 5. ALS vs Random Recommenders Performance Comparison

### Metrics functions

In [12]:
##different diversity metrics parsed form recall eval....object?
def get_ranking_results(ranking_eval):
    metrics = {
        "Precision@k": ranking_eval.precision_at_k(),
        "Recall@k": ranking_eval.recall_at_k(),
        "NDCG@k": ranking_eval.ndcg_at_k(),
        "Mean average precision": ranking_eval.map_at_k()
      
    }
    return metrics  

##different diversity metrics parsed form diversity eval....object?
def get_diversity_results(diversity_eval):
    metrics = {
        "catalog_coverage":diversity_eval.catalog_coverage(),
        "distributional_coverage":diversity_eval.distributional_coverage(), 
        "novelty": diversity_eval.novelty(), 
        "diversity": diversity_eval.diversity(), 
        "serendipity": diversity_eval.serendipity()
    }
    return metrics

In [13]:
def generate_summary(data, algo, k, ranking_metrics, diversity_metrics):
    summary = {"Data": data, "Algo": algo, "K": k}

    if ranking_metrics is None:
        ranking_metrics = {           
            "Precision@k": np.nan,
            "Recall@k": np.nan,            
            "nDCG@k": np.nan,
            "MAP": np.nan,
        }
    summary.update(ranking_metrics)
    summary.update(diversity_metrics)
    return summary

### ALS Recommender Performance Results

In [14]:
als_ranking_eval = SparkRankingEvaluation(
    test_df, 
    top_all, 
    k = TOP_K, 
    col_user="UserId", 
    col_item="MovieId",
    col_rating="Rating", 
    col_prediction="prediction",
    relevancy_method="top_k"
)

als_ranking_metrics = get_ranking_results(als_ranking_eval)

In [15]:
als_diversity_eval = SparkDiversityEvaluation(
    train_df = train_df, 
    reco_df = top_k_reco,
    col_user = COL_USER, 
    col_item = COL_ITEM
)

als_diversity_metrics = get_diversity_results(als_diversity_eval)

In [16]:
als_results = generate_summary(MOVIELENS_DATA_SIZE, "als", TOP_K, als_ranking_metrics, als_diversity_metrics)
print(als_results)

{'Data': '100k', 'Algo': 'als', 'K': 10, 'Precision@k': 0.04729586426299045, 'Recall@k': 0.01601477257161039, 'NDCG@k': 0.04309703774263865, 'Mean average precision': 0.004579045144199097, 'catalog_coverage': 0.385793018983466, 'distributional_coverage': 7.967257304554895, 'novelty': 11.659775792745016, 'diversity': 0.8922765632097469, 'serendipity': 0.8787329333952111}


## Random Recommender Performance Results

In [17]:
random_ranking_eval = SparkRankingEvaluation(
    test_df,
    pred_df,
    col_user=COL_USER,
    col_item=COL_ITEM,
    col_rating=COL_RATING,
    col_prediction="score",
    k=TOP_K,
)

random_ranking_metrics = get_ranking_results(random_ranking_eval)

In [19]:
random_diversity_eval = SparkDiversityEvaluation(
    train_df = train_df, 
    reco_df = pred_df, 
    col_user = COL_USER, 
    col_item = COL_ITEM
)
  
random_diversity_metrics = get_diversity_results(random_diversity_eval)

In [20]:
random_results = generate_summary(MOVIELENS_DATA_SIZE, "random", TOP_K, random_ranking_metrics, random_diversity_metrics)
print(random_results)

{'Data': '100k', 'Algo': 'random', 'K': 10, 'Precision@k': 0.015588547189819722, 'Recall@k': 0.005122908752536401, 'NDCG@k': 0.015613718941249626, 'Mean average precision': 0.0015506651228243207, 'catalog_coverage': 0.9957134109001837, 'distributional_coverage': 10.537609381993875, 'novelty': 12.151949574353871, 'diversity': 0.9235406134614499, 'serendipity': 0.8933719844556243}


## Result Comparison

In [21]:
cols = ["Data", "Algo", "K", "Precision@k", "Recall@k", "NDCG@k", "Mean average precision","catalog_coverage", "distributional_coverage","novelty", "diversity", "serendipity" ]
df_results = pd.DataFrame(columns=cols)

df_results.loc[1] = als_results 
df_results.loc[2] = random_results

In [22]:
df_results

,Data,Algo,K,Precision@k,Recall@k,NDCG@k,Mean average precision,catalog_coverage,distributional_coverage,novelty,diversity,serendipity
1,100k,als,10,0.047296,0.016015,0.043097,0.004579,0.385793,7.967257,11.659776,0.892277,0.878733
2,100k,random,10,0.015589,0.005123,0.015614,0.001551,0.995713,10.537609,12.151950,0.923541,0.893372


## 6. Calculate diversity metrics using item feature vector based item-item similarity

In the above section we calculate diversity metrics using item co-occurrence count based item-item similarity. In the scenarios when item features are available, we may want to calculate item-item similarity based on item feature vectors. In this section, we show how to calculate diversity metrics using item feature vector based item-item similarity.

That is content based- look at item-item feature similiarity

In [23]:
# Get movie features "title" and "genres"
movies = (
    data.groupBy("MovieId", "title", "genres").count()
    .na.drop()  # remove rows with null values
    .withColumn("genres", F.split(F.col("genres"), "\|"))  # convert to array of genres
    .withColumn("title", F.regexp_replace(F.col("title"), "[\(),:^0-9]", ""))  # remove year from title
    .drop("count")  # remove unused columns
)

In [24]:
# tokenize "title" column
title_tokenizer = Tokenizer(inputCol="title", outputCol="title_words")
tokenized_data = title_tokenizer.transform(movies)

# remove stop words
remover = StopWordsRemover(inputCol="title_words", outputCol="text")
clean_data = remover.transform(tokenized_data).drop("title", "title_words")

In [25]:
# convert text input into feature vectors

# step 1: perform HashingTF on column "text"
text_hasher = HashingTF(inputCol="text", outputCol="text_features", numFeatures=1024)
hashed_data = text_hasher.transform(clean_data)

# step 2: fit a CountVectorizerModel from column "genres".
count_vectorizer = CountVectorizer(inputCol="genres", outputCol="genres_features")
count_vectorizer_model = count_vectorizer.fit(hashed_data)
vectorized_data = count_vectorizer_model.transform(hashed_data)

# step 3: assemble features into a single vector
assembler = VectorAssembler(
    inputCols=["text_features", "genres_features"],
    outputCol="features",
)
feature_data = assembler.transform(vectorized_data).select("MovieId", "features")

feature_data.show(10, False)

+-------+---------------------------------------------+
|MovieId|features                                     |
+-------+---------------------------------------------+
|167    |(1043,[128,544,1025],[1.0,1.0,1.0])          |
|1343   |(1043,[38,300,1024],[1.0,1.0,1.0])           |
|1607   |(1043,[592,821,1024],[1.0,1.0,1.0])          |
|966    |(1043,[389,502,1028],[1.0,1.0,1.0])          |
|9      |(1043,[11,342,1014,1024],[1.0,1.0,1.0,1.0])  |
|1230   |(1043,[597,740,902,1025],[1.0,1.0,1.0,1.0])  |
|1118   |(1043,[702,1025],[1.0,1.0])                  |
|673    |(1043,[169,690,1027,1040],[1.0,1.0,1.0,1.0]) |
|879    |(1043,[909,1026,1027,1034],[1.0,1.0,1.0,1.0])|
|66     |(1043,[256,1025,1028],[1.0,1.0,1.0])         |
+-------+---------------------------------------------+
only showing top 10 rows



The features column is represented with a SparseVector object. For example, in the feature vector (1043,[128,544,1025],[1.0,1.0,1.0]), 1043 is the vector length, indicating the vector consisting of 1043 item features. The values at index positions 128,544,1025 are 1.0, and the values at other positions are all 0.

In [26]:
als_eval = SparkDiversityEvaluation(
    train_df = train_df, 
    reco_df = top_k_reco,
    item_feature_df = feature_data, 
    item_sim_measure="item_feature_vector",
    col_user = COL_USER, 
    col_item = COL_ITEM
)

als_diversity=als_eval.diversity()
als_serendipity=als_eval.serendipity()
print(als_diversity)
print(als_serendipity)

0.8738984131037538
0.8873467159479473


In [27]:
random_eval = SparkDiversityEvaluation(
    train_df = train_df, 
    reco_df = pred_df, 
    item_feature_df = feature_data, 
    item_sim_measure="item_feature_vector",    
    col_user = COL_USER, 
    col_item = COL_ITEM
)
  
random_diversity=random_eval.diversity()
random_serendipity=random_eval.serendipity()
print(random_diversity)
print(random_serendipity)

0.8986456855062992
0.8944051547993165


In [28]:
# cleanup spark instance
spark.stop()

## References
The metric definitions / formulations are based on the following references:

P. Castells, S. Vargas, and J. Wang, Novelty and diversity metrics for recommender systems: choice, discovery and relevance, ECIR 2011
G. Shani and A. Gunawardana, Evaluating recommendation systems, Recommender Systems Handbook pp. 257-297, 2010.
E. Yan, Serendipity: Accuracy’s unpopular best friend in recommender Systems, eugeneyan.com, April 2020
Y.C. Zhang, D.Ó. Séaghdha, D. Quercia and T. Jambor, Auralist: introducing serendipity into music recommendation, WSDM 2012